Import necessary libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import sagemaker
import boto3

Load the employee data

In [ ]:
data = pd.read_csv('employee_data.csv')
data

In [ ]:
# Rename the column
data.rename(columns={'Experience (Years)': 'Experience'}, inplace=True)

In [ ]:
data["Gender"]=data["Gender"].astype('string')
data["Position"]=data["Position"].astype('string')
data.dtypes


In [ ]:
#dropping ID column
# data.drop(columns=['ID'], inplace=True)

Split the data into training and test data sets (70% split)

In [ ]:
# train_data,test_data = train_test_split(data, test_size=0.3, random_state=42)

# # Display the number of rows in each dataset
# print(f'Total rows: {len(data)}')
# print(f'Training rows: {len(train_data)}')
# print(f'Testing rows: {len(test_data)}')

# # Save the split datasets into new CSV files
# train_data.to_csv('train_employee_data.csv', index=False)
# test_data.to_csv('test_employee_data.csv', index=False)

In [ ]:
# Apply one-hot encoding
data_encoded = pd.get_dummies(data,drop_first=True)

# Display the encoded DataFrame
data_encoded = data_encoded.astype(int)

print(data_encoded.shape)
print(data_encoded.columns)

In [ ]:
X=data_encoded.drop(columns="Salary")
X_columns=X.columns
print(X_columns)
Y=data_encoded["Salary"]



In [ ]:
#Splitting dataset in training and testing data

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=42)

In [ ]:
#Shape of testing and trainig data

X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

In [ ]:
#Standardising features for similar scale

scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)

X_test=scaler.transform(X_test)


print(X_train)
print("\n")
print(f"shape of trainig data is {X_train.shape}")
print("\n")
print('*'*80)
print("\n")
print(X_test)
print("\n")
print(f"shape of testing data is {X_test.shape}")

In [ ]:
# Convert the NumPy array to a DataFrame
X_train_df=pd.DataFrame(X_train,columns=X_columns)
X_train_df

In [ ]:
# Convert the NumPy array to a DataFrame
X_test_df=pd.DataFrame(X_test,columns=X_columns)
X_test_df

In [ ]:

Y_train_df=pd.DataFrame(Y_train)
Y_train_df

In [ ]:
Y_test_df=pd.DataFrame(Y_train)
Y_test_df

In [ ]:
# Save the split datasets into new CSV files
X_test_df.to_csv('X_test_df.csv', index=False)
X_train_df.to_csv('X_train_df.csv', index=False)
Y_test_df.to_csv('Y_test_df.csv', index=False)
Y_train_df.to_csv('Y_train_df.csv', index=False)

In [ ]:
#Multiple linear regression
linreg=LinearRegression()

#Fitting the model
linreg.fit(X_train,Y_train)

#Prediciting bike rental duration
y_pred=linreg.predict(X_test)

In [ ]:
X_test_df

In [ ]:
MAE=mean_absolute_error(Y_test, y_pred)
print(f'Mean Absolute Error of model is {round(MAE,2)}')

In [ ]:
X

In [ ]:
import numpy as np
new_data=X.sample(5)
model=linreg
new_data_scaled = scaler.transform(new_data)

# Make predictions using the model
predictions = model.predict(new_data_scaled)


#reversing transformations
original_data = scaler.inverse_transform(new_data_scaled)
original_ID=original_data[:,:1].astype(int)


# Convert array to DataFrame with a column name
df = pd.DataFrame({
    'ID': original_ID.flatten(),  # Flatten to match the (5,) shape
    'Predicted_Salary': predictions
})


# Print DataFrame
print(df)

In [ ]:

# categorical = ['Gender', 'Position']
# numerical = ['Experience (Years)']
# train_dicts = data[categorical + numerical].to_dict(orient='records')

# dv = DictVectorizer()
# X_train = dv.fit_transform(train_dicts)

# num_columns = X_train.shape[1]
# print("Number of columns in the feature matrix:", num_columns)

# target = 'Salary'
# Y_train = data[target].testues

# lr = LinearRegression()
# lr.fit(X_train, Y_train)

# y_pred = lr.predict(X_train)

# mean_squared_error(Y_train, y_pred, squared=False)

In [ ]:
# def pred_salaries(inputfile):
#     df_sample=pd.read_csv(inputfile)
#     sample_dicts = df_sample[categorical + numerical].to_dict(orient='records')
#     X_sample = dv.transform(sample_dicts)
#     pred_salaries = lr.predict(X_sample)
    
#     # Create a DataFrame with ID and predicted salaries
#     result_df = pd.DataFrame({
#         'ID': df_sample["ID"],
#         'Predicted_Salary': pred_salaries
#     })
    
#     return result_df


    

In [ ]:
# pred_salaries("./employee_data.csv")

In [ ]:
# ### Create a Pickle file using serialization
# import joblib

# # Assuming `classifier` is your trained classifier object

# # Save the classifier using joblib
# joblib.dump(lr, "regressor.pkl")

testidate model

In [ ]:

# df_test = pd.read_csv('./test_employee_data.csv')
# test_dicts = df_test[categorical + numerical].to_dict(orient='records')

# X_test = dv.transform(test_dicts)

# Y_test = df_test[target].testues

# y_pred = lr.predict(X_test)

# mean_squared_error(Y_test, y_pred, squared=False)


In [ ]:
# import pickle

# # Save the model
# with open('model.pkl', 'wb') as f:
#     pickle.dump(lr, f)


In [ ]:
# boto3_sm = boto3.client("sagemaker", region_name="ap-south-1")
# session = sagemaker.Session()
# region = session.boto_session.region_name
# bucket = "mlops-salary-predictor-app"
# print("Using Bucket:", bucket)

In [ ]:
# # Sagemaker takes training data from S3 bucket, so uploading data to S3 bucket
# train_path = session.upload_data(path="train_employee_data.csv", bucket=bucket)
# test_path = session.upload_data(path="test_employee_data.csv", bucket=bucket)

In [ ]:
# %%writefile script.py

# import argparse
# import os
# import json
# import sklearn
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
# import joblib
# import pathlib
# from io import StringIO
# import boto3
# import pandas as pd
# import numpy as np

# def model_fn(model_dir):
#     clf = joblib.load(os.path.join(model_dir, "model.joblib"))
#     return clf

# if __name__ =='__main__':

#     print("[INFO] Extracting arguments")
#     parser = argparse.ArgumentParser()

#     # hyperparameters sent by the client are passed as command-line arguments to the script.
#     parser.add_argument('--n_estimators', type=int, default=100)
#     parser.add_argument('--random_state', type=int, default=0)
#     # parser.add_argument('--epochs', type=int, default=10)
#     # parser.add_argument('--batch-size', type=int, default=100)
#     # parser.add_argument('--learning-rate', type=float, default=0.1)

#     # an alternative way to load hyperparameters via SM_HPS environment variable.
#     # parser.add_argument('--sm-hps', type=json.loads, default=os.environ['SM_HPS'])

#     # input data and model directories
#     parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
#     parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
#     parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
#     parser.add_argument('--train_file', type=str, default='train_employee_data.csv')
#     parser.add_argument('--test_file', type=str, default='test_employee_data.csv')

#     args, _ = parser.parse_known_args()

#     print("SKLearn Version: ", sklearn.__version__)
#     print("Joblib Version: ", joblib.__version__)

#     print("[INFO] Reading data")
#     print()
#     train_df = pd.read_csv(os.path.join(args.train, args.train_file))
#     test_df = pd.read_csv(os.path.join(args.test, args.test_file))

#     features = list(train_df.columns)
#     # label = features.pop(-1)
    
#     print("Building training and testing datasets")
#     print()
#     X_train = train_df[features]
#     X_test = test_df[features]
#     Y_train = train_df[label]
#     Y_test = test_df[label]

#     print('Column order: ')
#     print(features)
#     print()
    
#     print("Label column is: ",label)
#     print()
    
#     print("Data Shape: ")
#     print()
#     print("---- SHAPE OF TRAINING DATA (70%) ----")
#     print(X_train.shape)
#     print(Y_train.shape)
#     print()
#     print("---- SHAPE OF TESTING DATA (30%) ----")
#     print(X_test.shape)
#     print(Y_test.shape)
#     print()
    
#     print("Training RandomForest Model.....")
#     print()
#     model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=None)
#     model.fit(X_train, Y_train)
#     print()

#     model_path = os.path.join(args.model_dir, "model.joblib")
#     joblib.dump(model, model_path)
#     print("Model persisted at " + model_path)
#     print()

    
#     y_pred_test = model.predict(X_test)
#     test_acc = accuracy_score(Y_test,y_pred_test)
#     test_rep = classification_report(Y_test,y_pred_test)

#     print()
#     print("---- METRICS RESULTS FOR TESTING DATA ----")
#     print()
#     print("Total Rows are: ", X_test.shape[0])
#     print('[TESTING] Model Accuracy is: ', test_acc)
#     print('[TESTING] Testing Report: ')
#     print(test_rep)